## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

In [ ]:
import numpy as np
from numba import njit
import pandas as pd

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.
Задание выполнить несколькими способами ("naive" Python, numpy, с использованием numba декоратора @njit) Проверить время выполнения каждого способа %time или %%time

In [ ]:
A = np.random.randint(1, 1000, size=1_000_000)

In [ ]:
# Способ 1: "naive" Python
%%time
B = []
for n in A:
    B.append(n + 100)
sum(B) / len(B)

UsageError: Line magic function `%%time` not found.


In [ ]:
# Способ 2: numpy
%time
B = A + 100
B.mean()

CPU times: user 10 µs, sys: 4 µs, total: 14 µs
Wall time: 26 µs


599.622941

In [ ]:
@njit
def naive(A):
    B = []
    for n in A:
        B.append(n + 100)
    return sum(B) / len(B)

In [ ]:
# Способ 3: @njit
%time
naive(A)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


599.622941

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв. Задание выполнить несколькими способами ("naive" Python, numpy, с использованием numba декоратора @njit - оценить возможность). Проверить время выполнения каждого способа %time или %%time

In [ ]:
df = pd.DataFrame()
df['1'] = np.random.randint(1, 5000, size=2_000_000)
df['2'] = np.random.randint(1, 5000, size=2_000_000)
df['3'] = np.random.randint(1, 5000, size=2_000_000)
df['4'] = np.random.randint(1, 5000, size=2_000_000)
df['key'] = [chr(w) for w in np.random.randint(97, 122, size=2_000_000)]
df[:8]

,1,2,3,4,key
0,1192,479,765,704,r
1,244,1758,859,2877,f
2,4603,2851,4276,2622,b
3,4960,4358,1515,1515,w
4,629,1200,1222,1310,u
5,1276,4234,1601,1466,f
6,4754,4039,1188,3885,w
7,633,3258,3809,1654,p


In [ ]:
# Способ 1: "naive" Python
%%time
res = []
letters = 'abcde'
for line in df.index:
    if df.iloc[line]['key'] in letters:
        res.append(line)

df.iloc[res]

UsageError: Line magic function `%%time` not found.


In [ ]:
# Способ 2: numpy
df[df['key'] in []]

## Лабораторная работа 3

In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
from datetime import datetime

In [ ]:
recipes = pd.read_csv('recipes_sample.csv')
reviews = pd.read_csv('reviews_sample.csv', index_col=0)
reviews['date'] = pd.to_datetime(reviews['date'])
reviews.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [ ]:
# Способ А
def foo1():
  rating_sum = 0
  count_rating = 0
  for index, row in reviews.iterrows():
    if row['date'].year == 2010:
      count_rating += 1
      rating_sum += row['rating']
  return rating_sum / count_rating

In [ ]:
%%time
foo1()

CPU times: user 9.31 s, sys: 46.7 ms, total: 9.36 s
Wall time: 10 s


4.4544402182900615

In [ ]:
# Способ Б
def foo2():
  rating_sum = 0
  count_rating = 0
  reviews_2010 = reviews[(datetime(2010, 1, 1) <= reviews['date']) & (reviews['date'] <= datetime(2010, 12, 31))]
  for index, row in reviews_2010.iterrows():
    count_rating += 1
    rating_sum += row['rating']
  return(rating_sum / count_rating)

In [ ]:
%%time
foo2()

CPU times: user 695 ms, sys: 4.74 ms, total: 700 ms
Wall time: 704 ms


4.4544402182900615

In [ ]:
# Способ В
def foo3():
  return reviews[(datetime(2010, 1, 1) <= reviews['date']) & (reviews['date'] <= datetime(2010, 12, 31))]['rating'].mean()

In [ ]:
%%time
foo3()

CPU times: user 10.4 ms, sys: 0 ns, total: 10.4 ms
Wall time: 13.3 ms


4.4544402182900615

2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [ ]:
# Модификация функции 1Б
%%time
sum(reviews[(datetime(2010, 1, 1) <= reviews['date']) & (reviews['date'] <= datetime(2010, 12, 31))]['rating']) / len(reviews[(datetime(2010, 1, 1) <= reviews['date']) & (reviews['date'] <= datetime(2010, 12, 31))])

CPU times: user 23.1 ms, sys: 1.79 ms, total: 24.9 ms
Wall time: 46.3 ms


4.4544402182900615

In [ ]:
from line_profiler import LineProfiler
lp = LineProfiler()
lp_wrapper = lp(foo1)()
lp.print_stats()

Timer unit: 1e-09 s

Total time: 15.3247 s
File: <ipython-input-25-145a2a914d49>
Function: foo1 at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def foo1():
     3         1       1570.0   1570.0      0.0    rating_sum = 0
     4         1        385.0    385.0      0.0    count_rating = 0
     5    126697        1e+10 106579.0     88.1    for index, row in reviews.iterrows():
     6    126696 1702852406.0  13440.5     11.1      if row['date'].year == 2010:
     7     12094    6935571.0    573.5      0.0        count_rating += 1
     8     12094  111652047.0   9232.0      0.7        rating_sum += row['rating']
     9         1        897.0    897.0      0.0    return rating_sum / count_rating



Больше всего времени затрачивается на работу цикла for (88% времени). Достаточно много времени еще уходит на проверку даты (11% времени работы функции)

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [ ]:
%%time
get_word_reviews_count(reviews)

CPU times: user 31.7 s, sys: 401 ms, total: 32.1 s
Wall time: 32.5 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [ ]:
lp = LineProfiler()
lp_wrapper = lp(get_word_reviews_count)(reviews)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 62.124 s
File: <ipython-input-41-b1bc049bcd0c>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count(df):
     2         1       1702.0   1702.0      0.0      word_reviews = {}
     3    126680        2e+10 129385.2     26.4      for _, row in df.dropna(subset=['review']).iterrows():
     4    126679 3078746082.0  24303.5      5.0          recipe_id, review = row['recipe_id'], row['review']
     5    126679  811010039.0   6402.1      1.3          words = review.split(' ')
     6   6918689 1930343119.0    279.0      3.1          for word in words:
     7   6792010 4401340360.0    648.0      7.1              if word not in word_reviews:
     8    174944  125102605.0    715.1      0.2                  word_reviews[word] = []
     9   6792010 5480204581.0    806.9      8.8              word_reviews[word].append(recipe_id)
    10  

больше всего времени потрачено на обход всех строк таблицы (цикл for). Функция дважды обходит строки таблицы. На это уходит 52% времени работы функции

In [ ]:
from collections import Counter

In [ ]:
def get_word_reviews_count_2(df):
  words = ' '.join(reviews.dropna(subset=['review'])['review'].values).split(' ')
  return Counter(words)

In [ ]:
%%time
get_word_reviews_count_2(reviews)

CPU times: user 1.89 s, sys: 109 ms, total: 2 s
Wall time: 1.99 s


Counter({'Last': 94,
         'week': 804,
         'whole': 5628,
         'sides': 312,
         'of': 109029,
         'frozen': 2647,
         'salmon': 729,
         'fillet': 60,
         'was': 88781,
         'on': 34583,
         'sale': 149,
         'in': 61539,
         'my': 44144,
         'local': 561,
         'supermarket,': 10,
         'so': 46090,
         'I': 285147,
         'bought': 1369,
         'tons': 161,
         '(okay,': 5,
         'only': 13965,
         '3,': 48,
         'but': 42513,
         'total': 381,
         'weight': 160,
         'over': 9065,
         '10': 2303,
         'pounds).': 2,
         '': 214145,
         'This': 39448,
         'recipe': 41098,
         'is': 55075,
         'perfect': 4398,
         'for': 121224,
         'fillet,': 14,
         'even': 7878,
         'though': 2314,
         'it': 111175,
         'calls': 520,
         'steaks.': 93,
         'cut': 6688,
         'up': 13585,
         'the': 266050,
     

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта.
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
df1 = reviews[reviews['rating'] != 0]
df1.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [ ]:
def foo1(df, id):
  res = 0
  mean_val = 0
  values = []
  for index, val in df.iterrows():
    if val['recipe_id'] == id:
      values.append(val['rating'])
      mean_val += val['rating']
  mean_val /= len(values)
  for v in values:
    res += abs(v - mean_val) / v
  return (res * 100) / len(values)

In [ ]:
%%time
foo1(df1, 21752)

CPU times: user 9.06 s, sys: 49.9 ms, total: 9.11 s
Wall time: 9.26 s


86.328125

In [ ]:
from numba import njit
import numpy as np

def foo2(df, id):
    res = 0
    mean_val = 0.0
    values = []
    for index, val in df.iterrows():
        if val['recipe_id'] == id:
            values.append(val['rating'])
            mean_val += val['rating']
    mean_val /= len(values)
    values = np.array(values)
    for v in values:
        res += abs(v - mean_val) / v
    return (res * 100) / len(values)

In [ ]:
%%time
foo2(df1, 21752)

CPU times: user 8.76 s, sys: 49.1 ms, total: 8.81 s
Wall time: 9 s


86.328125

In [ ]:
def foo3(df, id):
  df2 = df[df['recipe_id'] == id]
  mean_val = df2['rating'].mean()
  res = ((df2['rating'] - mean_val).abs()/df2['rating']).sum()
  return (res * 100) / df2.shape[0]

In [ ]:
%%time
foo3(df1, 21752)

CPU times: user 5.42 ms, sys: 0 ns, total: 5.42 ms
Wall time: 8.74 ms


86.328125

In [ ]:
@njit
def foo4(df):
  mean_val = np.mean(df)
  res = np.sum(np.abs(df - mean_val)/df)
  return (res * 100) / len(df)

In [ ]:
%%time
ratings = np.array(df1[df1['recipe_id'] == 21752]['rating'])
foo4(ratings)

CPU times: user 635 ms, sys: 1.55 ms, total: 636 ms
Wall time: 653 ms


86.328125